In [2]:
import pandas as pd
import pickle as pickle
import ast

from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, Trainer, TrainingArguments
from omegaconf import OmegaConf
from dataset_utils import load_data, label_to_num, tokenized_dataset
from datasets import RE_Dataset
from metrics import compute_metrics

import numpy as np
import argparse
import random
import torch

In [2]:
def add_typed_entity_marker_punct(file_path):
    """각 엔티티 토큰은 @, *, &, ^로 고정. 다만 논문과 다르게 우리가 사용할 한글 토크나이저에
       1. 엔티티 타입이 없을 수 있어 [UNK]으로 되는것을 방지하고자 엔티티 타입 자체를 스페셜 토큰으로 추가
       2. wordpiece 토크나이저를 사용할 수 있으므로 object entity의 토큰을 논문의 '#'대신 '&'로 대체사용 
   
    Args:
        file_path (str): csv파일경로

    Returns:
        list: 스페셜 토큰에 추가할 엔티티 타입의 list
    """
    entity_tokens = set()
    df = pd.read_csv(file_path)

    result = []

    for idx, row in df.iterrows():
        sentence = row['sentence']
        subject_entity = ast.literal_eval(row['subject_entity'])
        object_entity = ast.literal_eval(row['object_entity'])

        new_sentence = ''

        curr_entity_tokens = [subject_entity["type"],
                              object_entity["type"]]
        entity_tokens.update(curr_entity_tokens)

        if subject_entity['start_idx'] < object_entity['start_idx']:
            new_sentence += sentence[ :subject_entity['start_idx'] ]
            new_sentence += f"@*{curr_entity_tokens[0]}*{sentence[subject_entity['start_idx']:subject_entity['end_idx']+1]}@"
            new_sentence += sentence[subject_entity['end_idx']+1:object_entity['start_idx']]
            new_sentence += f"&^{curr_entity_tokens[1]}^{sentence[object_entity['start_idx']:object_entity['end_idx']+1]}&"
            new_sentence += sentence[object_entity['end_idx']+1:]
        else:
            new_sentence += sentence[ :object_entity['start_idx'] ]
            new_sentence += f"&^{curr_entity_tokens[1]}^{sentence[object_entity['start_idx']:object_entity['end_idx']+1]}&"
            new_sentence += sentence[object_entity['end_idx']+1:subject_entity['start_idx']]
            new_sentence += f"@*{curr_entity_tokens[0]}*{sentence[subject_entity['start_idx']:subject_entity['end_idx']+1]}@"
            new_sentence += sentence[subject_entity['end_idx']+1:]
        result.append(new_sentence)
    
    df['sentence'] = result
    df.to_csv('typed_entity_marker_punct_train.csv')

    return list(entity_tokens) 


In [3]:
df = pd.read_csv("../dataset/train/train_final.csv")

In [7]:
df.object_type.unique()

array(['PER', 'ORG', 'DAT', 'LOC', 'POH', 'NOH'], dtype=object)

In [ ]:
'[PER]', '[ORG]', '[DAT]', '[LOC]', '[POH]', '[NOH]'

In [8]:
def preprocessing_dataset(dataset):
  """ 처음 불러온 csv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다."""
  subject_entity = []
  object_entity = []
  for i,j in zip(dataset['subject_entity'], dataset['object_entity']):
    i = i[1:-1].split(',')[0].split(':')[1]
    j = j[1:-1].split(',')[0].split(':')[1]

    subject_entity.append(i)
    object_entity.append(j)
  out_dataset = pd.DataFrame({'id':dataset['id'], 'sentence':dataset['sentence'],'subject_entity':subject_entity,'object_entity':object_entity,'label':dataset['label'],})
  return out_dataset

In [9]:
def tokenized_dataset(dataset, tokenizer):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02 in zip(dataset['subject_entity'], dataset['object_entity']):
    temp = ''
    temp = e01 + '[SEP]' + e02
    concat_entity.append(temp)
  tokenized_sentences = tokenizer(
      concat_entity,
      list(dataset['sentence']),
      return_tensors="pt",
      padding=True,
      truncation=True,
      max_length=256,
      add_special_tokens=True,
      )
  return tokenized_sentences

In [11]:
def load_data(dataset_dir):
  """ csv 파일을 경로에 맡게 불러 옵니다. """
  pd_dataset = pd.read_csv(dataset_dir)
  dataset = preprocessing_dataset(pd_dataset)
  
  return dataset

def load_test_dataset(dataset_dir, tokenizer):
  """
    test dataset을 불러온 후,
    tokenizing 합니다.
  """
  test_dataset = load_data(dataset_dir)
  test_label = list(map(int,test_dataset['label'].values))
  # tokenizing dataset
  tokenized_test = tokenized_dataset(test_dataset, tokenizer)
  return test_dataset['id'], tokenized_test, test_label

def label_to_num(label):
  num_label = []
  with open('./pickle/dict_label_to_num.pkl', 'rb') as f:
    dict_label_to_num = pickle.load(f)
  for v in label:
    num_label.append(dict_label_to_num[v])
  
  return num_label

def num_to_label(label):
  """
    숫자로 되어 있던 class를 원본 문자열 라벨로 변환 합니다.
  """
  origin_label = []
  with open('./pickle/dict_num_to_label.pkl', 'rb') as f:
    dict_num_to_label = pickle.load(f)
  for v in label:
    origin_label.append(dict_num_to_label[v])
  
  return origin_label


In [42]:
  MODEL_NAME = 'klue/bert-base'
  tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [13]:

  # load dataset
  train_dataset = load_data("../dataset/train/train.csv")


In [16]:

  train_label = label_to_num(train_dataset['label'].values)


In [18]:
tokenized_train = tokenized_dataset(train_dataset, tokenizer)

# make dataset for pytorch.
RE_train_dataset = RE_Dataset(tokenized_train, train_label)

In [25]:
RE_train_dataset[1].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [26]:
tokenized_train

{'input_ids': tensor([[    2,    11, 29830,  ...,     0,     0,     0],
        [    2,    11,  3772,  ...,     0,     0,     0],
        [    2,    11,  4104,  ...,     0,     0,     0],
        ...,
        [    2,    11, 18272,  ...,     0,     0,     0],
        [    2,    11, 15710,  ...,     0,     0,     0],
        [    2,    11, 15437,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [27]:
  model_config =  AutoConfig.from_pretrained(MODEL_NAME)


In [28]:
model_config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

In [3]:
with open("../dataset/train/train_final.csv", "r") as fh:
    data = pd.read_csv(fh)


In [13]:
d=next(data.iterrows())[1]

In [11]:
    def tokenize(self, sentence, subj_type, obj_type, ss, se, os, oe):
        sents = []
        subj_type , obj_type = f"[{subj_type}]", f"[{obj_type}]"
        tokens = self.tokenizer.tokenize(sentence)
        for i_t, token in enumerate(tokens):
            if i_t == ss:
                new_ss = len(sents)
                sents.extend(['@'] + ['*'] + [subj_type] + ['*'] + [token])
            if i_t == se:
                sents.extend([token] + ['@'])
            if i_t == os:
                new_os = len(sents)
                sents.extend(["#"] + ['^'] + [obj_type] + ['^'] + [token])
            if i_t == oe:
                sents.extend([token] + ['#'])
        sents = sents[:self.args.model.max_seq_length - 2]
        input_ids = self.tokenizer.convert_tokens_to_ids(sents)
        input_ids = self.tokenizer.build_inputs_with_special_tokens(input_ids)
        
        return input_ids, new_ss + 1, new_os + 1


In [27]:
ss, se = d['subject_start_idx'], d['subject_end_idx']
os, oe = d['object_start_idx'], d['object_end_idx']
subj_type=d['subject_type']
obj_type=d['object_type']
sentence=d['sentence']

In [19]:
        subj_type , obj_type = f"[{subj_type}]", f"[{obj_type}]"


In [24]:
tokens=tokenizer.tokenize(sentence)

In [34]:
tokens

['〈',
 'So',
 '##me',
 '##th',
 '##ing',
 '〉',
 '는',
 '조지',
 '해리',
 '##슨',
 '##이',
 '쓰',
 '##고',
 '비틀즈',
 '##가',
 '1969',
 '##년',
 '앨범',
 '《',
 'Ab',
 '##be',
 '##y',
 'Ro',
 '##ad',
 '》',
 '에',
 '담',
 '##은',
 '노래',
 '##다',
 '.']

In [40]:
sents=[]
for i_t, token in enumerate(tokens):
    if i_t == ss:
        new_ss = len(sents)
        sents.extend(['@'] + ['*'] + [subj_type] + ['*'])
    if i_t == se:
        sents.extend(['@'])
    if i_t == os:
        new_os = len(sents)
        sents.extend(["#"] + ['^'] + [obj_type] + ['^'])
    if i_t == oe:
        sents.extend(['#'])
    sents.append(token)

In [43]:
sents

['〈',
 'So',
 '##me',
 '##th',
 '##ing',
 '〉',
 '는',
 '조지',
 '해리',
 '##슨',
 '##이',
 '쓰',
 '##고',
 '#',
 '^',
 'PER',
 '^',
 '비틀즈',
 '##가',
 '1969',
 '##년',
 '앨범',
 '#',
 '《',
 'Ab',
 '##be',
 '##y',
 'Ro',
 '##ad',
 '@',
 '*',
 'ORG',
 '*',
 '》',
 '에',
 '@',
 '담',
 '##은',
 '노래',
 '##다',
 '.']

In [41]:
print(ss,se,os,oe)

24 26 13 18


In [46]:
sentence1= '〈Something〉는 #^[PER]^조지 해리슨#이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.'

In [47]:
tokenizer.tokenize(sentence1)

['〈',
 'So',
 '##me',
 '##th',
 '##ing',
 '〉',
 '는',
 '#',
 '^',
 '[',
 'PER',
 ']',
 '^',
 '조지',
 '해리',
 '##슨',
 '#',
 '이',
 '쓰',
 '##고',
 '비틀즈',
 '##가',
 '1969',
 '##년',
 '앨범',
 '《',
 'Ab',
 '##be',
 '##y',
 'Ro',
 '##ad',
 '》',
 '에',
 '담',
 '##은',
 '노래',
 '##다',
 '.']

In [42]:
new_ss

29

In [ ]:
            input_ids, new_ss, new_os = self.tokenize(sentence, d['subject_type'], d['object_type'], ss, se, os, oe)
            rel = self.LABEL_TO_ID[d['label']]

            feature = {
                'input_ids': input_ids,
                'labels': rel,
                'ss': new_ss,
                'os': new_os,
            }
            features.append(feature)

In [32]:

for idx, d in df.iterrows():
    s=count_word_in_text(d['sentence'], d["subject_word"])
    o=count_word_in_text(d['sentence'], d["object_word"])
    if s!= 1 or o!=1:
        print(s,o)
        print(d['sentence'], d["subject_word"],d["object_word"])
        

2 2
중공군에게 온전히 대항할 수 없을 정도로 약해진 국민당은 타이베이로 수도를 옮기는 것을 결정해, 남아있는 중화민국군의 병력이나 국가, 개인의 재산등을 속속 타이완으로 옮기기 시작해, 12월에는 중앙 정부 기구도 모두 이전해 타이베이 시를 중화민국의 새로운 수도로 삼았다. 중화민국 타이베이
1 2
2009년 9월, 미국 프로 야구 필라델피아 필리스 소속의 야구 선수 박찬호는 《MBC 스페셜-박찬호는 당신을 잊지 않았다》 편에서 “최진실 씨의 아픔과 죽음의 고통을 이해합니다. 최진실 씨 사건에 눈물을 흘렸습니다. 저도 죽으려고 마음을 먹었던 적이 있었습니다. 잘하려고 애를 쓰는데 비난과 비판이 쏟아졌습니다. 머리가 빠지고 너무 힘들었습니다”라고 말하며 최진실의 죽음에 대해 안타까움을 표현했다. 필라델피아 필리스 박찬호
1 2
문성민은 경기대학교에 입학하여 황동일, 신영석과 함께 경기대학교의 전성기를 이끌면서 하계대회, 전국체전, 최강전 등 3관왕을 이룬다. 문성민 경기대
2 1
스포츠계에서도 막강한 영향력을 펼치고 있어 2008년 9월 만수르 빈 자이드 알나하얀 소유의 에 맨체스터 시티 FC의 인수를 제안했고 인수 완료 후 잉글랜드 프리미어리그의 맨체스터 시티 FC의 구단주가 되었다. 맨체스터 시티 FC 프리미어리그
1 3
양당이 모두 해체한 뒤 신당을 창당하는 것이 아닌 유신당이 해체 뒤 기존의 민주당에 합류하는 방식이기 때문에, 옛 모두의 당 시절 비례대표로 선출되어 두레당을 거쳐 유신당에 합류했던 참의원 의원 5명(오노 지로, 가와다 류헤이, 시바타 다쿠미, 데라타 스케시로, 마야마 유이치)은 국회법 제109조의 2의 규정에 의해 신당에 입당할 경우 의원직이 자동 박탈되기 때문에 유신당 해체와 함께 임기 만료 전까지 무소속 신분을 유지하게 되었다. 두레당 유신당
2 2
새정치국민회의는 한나라당 내 민주계 의원들을, 자유민주연합은 민정계 의원들을 각각 접촉하여 반란 표를 부탁했으며, 한나라당은 반대로 자유민주연합 내 충청권 의원들을 접촉하여 "국회의장을

KeyboardInterrupt: 

In [6]:
df

,Unnamed: 0,id,sentence,label,source,subject_word,subject_start_idx,subject_end_idx,subject_type,object_word,object_start_idx,object_end_idx,object_type,entity_pair
0,0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,no_relation,wikipedia,비틀즈,24,26,ORG,조지 해리슨,13,18,PER,ORG-PER
1,1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,no_relation,wikitree,민주평화당,19,23,ORG,대안신당,14,17,ORG,ORG-ORG
2,2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,org:member_of,wikitree,광주FC,21,24,ORG,한국프로축구연맹,34,41,ORG,ORG-ORG
3,3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,org:top_members/employees,wikitree,아성다이소,13,17,ORG,박정부,22,24,PER,ORG-PER
4,4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,no_relation,wikipedia,요미우리 자이언츠,22,30,ORG,1967,0,3,DAT,ORG-DAT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32465,32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,per:employee_of,wikitree,유기준,93,95,PER,부산 서구·동구,100,107,LOC,PER-LOC
32466,32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...",per:colleagues,wikipedia,최시형,7,9,PER,손병희,17,19,PER,PER-PER
32467,32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,org:top_members/employees,wikitree,완도군,0,2,ORG,신우철,7,9,PER,ORG-PER
32468,32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...",no_relation,wikipedia,JTBC,6,9,ORG,중앙홀딩스,21,25,ORG,ORG-ORG


In [7]:
import re

In [27]:
def count_word_in_text(text, word):
    matches =re.findall(re.escape(word), text)
    return len(matches)

In [25]:
text = "이 문장 안에는 여러 단어가 있으며, 특정 단어를 찾을 수 있습니다. 예를 들어, '단어'라는 단어를 찾아보세요."
word = "단어"

In [26]:
count_word_in_text(text, word)

4

In [39]:
tokenizer.tokenize("내가 너에게 가는 데까지 10분을 소요했다.")

['내',
 '##가',
 '너',
 '##에',
 '##게',
 '가',
 '##는',
 '데',
 '##까',
 '##지',
 '10',
 '##분',
 '##을',
 '소요',
 '##했',
 '##다',
 '.']

In [56]:
tokenizer.tokenize("내가 너에게 가는 데까지 10분을 소요했다.")

['내',
 '##가',
 '너',
 '##에',
 '##게',
 '가',
 '##는',
 '데',
 '##까',
 '##지',
 '10',
 '##분',
 '##을',
 '소요',
 '##했',
 '##다',
 '.']

In [41]:
MODEL_NAME1 = 'xlm-roberta-large'
tokenizer2 = AutoTokenizer.from_pretrained(MODEL_NAME1)

In [61]:
tokens =[]
for t in text.split():
    tokens.extend(tokenizer.tokenize(t))
tokens1=tokenizer.tokenize(text)
tokens1 == tokens

True

In [64]:
1. split을 한다.
2. for word in words:
3. cs = 0
4. if cs == ss: # tokeni
5. elif cs == se: 
6. elif cs == os:
7. elif cs == oe:
5. else : cs += len(word) + 1

In [70]:
example=next(df.iterrows())[1]

In [72]:
sentence=example['sentence']

In [84]:
sentence

'〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.'

In [78]:
sentence.split()

['〈Something〉는',
 '조지',
 '해리슨이',
 '쓰고',
 '비틀즈가',
 '1969년',
 '앨범',
 '《Abbey',
 'Road》에',
 '담은',
 '노래다.']

In [83]:
tokenizer.tokenize(sentence.split()[1])

['조지']

In [81]:
len(sentence.split()[0])+1

13

In [82]:
sentence[13]

'조'

In [ ]:
for i in range(len(list1) - len(list2) + 1):
    if list1[i:i + len(list2)] == list2:
        index = i
        break

index


In [ ]:
words = sentence.split()
0. subject/object entity -> tokenize sbj tokens
1. words에서 어느 부분이 sub 부분 인덱스고 - > (sws, swe), obj 부분 인덱스 - > (ows, owe) 인지 확인
2. for idx, word in enumerate(words):
      

3.  
        if idx not in {sws,swe,ows,owe}:
            sents.extend(tokenize(word))
        else:
            if idx == sws:
                if sws == swe:
                    tokenize(word)에서 subj_tokens의 위치를 파악
                    위치에 ['@'] + ['*'] + [subj_type] + ['*'] + subj_tokens + ['@']
                else:
                    위치에 ['@'] + ['*'] + [subj_type] + ['*'] + subj_tokens

                sents.extend()

            elif idx == swe:
                tokenize(word)에서 subj_tokens의 위치를 파악
                위치에  subj_tokens + ['@']
                sents.extend()

            if idx == ows:
                if ows == owe:
                    tokenize(word)에서 obj_tokens의 위치를 파악
                    위치에 ["#"] + ['^'] + [obj_type] + ['^'] + obj_tokens + ["#"]
                else:
                    위치에 ["#"] + ['^'] + [obj_type] + ['^'] + obj_tokens + ["#"]

                sents.extend()

            elif idx == owe:
                tokenize(word)에서 obj_tokens의 위치를 파악
                위치에  obj_tokens + ["#"]
                sents.extend()

        

        

1. ss 또는 os 가 나오기 전까지 tokenize -> sents 에 extend
2. ss 또는 os가 나왔음. 그러면 se 또는 oe가 나오는 word list -> word list 1 와 se 또는 oe가 나오지 않는 word list -> word list 2 를 구분함.
3. word list 1과 subject entity를 각각 tokenize
4. word list 1에서 subject entity의 토큰의 위치 탐색 + 이때 len(sents) + 시작 위치를 new_ss로 입력
5. 그 위치에 subject entity 토큰에 추가한 것을 추가 -> 최종 list -> sents.extend(최종 list)
6. word list 2 전까지 값을 cs라 입력


7. cs부터 다시 ss 또는 os가 나오기 전까지 tokenize -> sents 에 extend
7. word list 2 에서 

In [128]:
example

Unnamed: 0                                                           0
id                                                                   0
sentence             〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...
label                                                      no_relation
source                                                       wikipedia
subject_word                                                       비틀즈
subject_start_idx                                                   24
subject_end_idx                                                     26
subject_type                                                       ORG
object_word                                                     조지 해리슨
object_start_idx                                                    13
object_end_idx                                                      18
object_type                                                        PER
entity_pair                                                    ORG-PER
Name: 

In [130]:
def word_idx_extract(words, ns, ne):
    
    word_indices = []
    start_index = 0

    for word in words:
        end_index = start_index + len(word) - 1
        word_indices.append((start_index, end_index))
        start_index = end_index + 2

    word_idx=[]
    for i, (start, end) in enumerate(word_indices):
        if ns in range(start, end + 1) or ne in range(start, end + 1):
            word_idx.append(i)
            
    return word_idx[0] , word_idx[-1]

In [132]:
def token_location(list1, list2):
    for i in range(len(list1) - len(list2) + 1):
        if list1[i:i + len(list2)] == list2:
            index = i
            return i, i+len(list2)-1


In [146]:
    sws, swe = word_idx_extract(words, ss,se)
    ows, owe = word_idx_extract(words, os,oe)

    subj_tokens= tokenizer.tokenize(example['subject_word'])
    obj_tokens= tokenizer.tokenize(example['object_word'])
    subj_type, obj_type = example['subject_type'], example['object_type']

In [153]:
from tqdm import tqdm

In [154]:
class Processor:
    def __init__(self, args, tokenizer):
        super().__init__()
        self.args = args
        self.tokenizer = tokenizer
        self.new_tokens = ['[PER]', '[ORG]', '[DAT]', '[LOC]', '[POH]', '[NOH]']
        self.tokenizer.add_tokens(self.new_tokens)
        self.LABEL_TO_ID = {'no_relation': 0, 'org:top_members/employees': 1, 'org:members': 2, 'org:product': 3, 'per:title': 4, 'org:alternate_names': 5, 'per:employee_of': 6, \
                'org:place_of_headquarters': 7, 'per:product': 8, 'org:number_of_employees/members': 9, 'per:children': 10, 'per:place_of_residence': 11, 'per:alternate_names': 12, \
                'per:other_family': 13, 'per:colleagues': 14, 'per:origin': 15, 'per:siblings': 16, 'per:spouse': 17, 'org:founded': 18, 'org:political/religious_affiliation': 19, \
                'org:member_of': 20, 'per:parents': 21, 'org:dissolved': 22, 'per:schools_attended': 23, 'per:date_of_death': 24, 'per:date_of_birth': 25, 'per:place_of_birth': 26, \
                'per:place_of_death': 27, 'org:founded_by': 28, 'per:religion': 29}
        
    def word_idx_extract(self, words, ns, ne):
        
        word_indices = []
        start_index = 0

        for word in words:
            end_index = start_index + len(word) - 1
            word_indices.append((start_index, end_index))
            start_index = end_index + 2

        word_idx=[]
        for i, (start, end) in enumerate(word_indices):
            if ns in range(start, end + 1) or ne in range(start, end + 1):
                word_idx.append(i)
                
        return word_idx[0] , word_idx[-1]


    def token_location(self, list1, list2):
        for i in range(len(list1) - len(list2) + 1):
            if list1[i:i + len(list2)] == list2:
                index = i
                return i, i+len(list2)-1

    def tokenize(self, sentence, subject_word, object_word, subj_type, obj_type, ss, se, os, oe):
        
        words = sentence.split()

        sws, swe = self.word_idx_extract(words, ss,se)
        ows, owe = self.word_idx_extract(words, os,oe)

        subj_tokens= self.tokenizer.tokenize(subject_word)
        obj_tokens= self.tokenizer.tokenize(object_word)

        sents =[]
        subj_type , obj_type = f"[{subj_type}]", f"[{obj_type}]"
        subj_token_collect = []
        obj_token_collect = []

        for idx, word in enumerate(words):
            tokens = tokenizer.tokenize(word)
            if idx not in range(sws,swe+1) and idx not in range(ows,owe+1):
                sents.extend(tokens)
            else:
                if sws <= idx and idx <= swe:
                    subj_token_collect.extend(tokens)
                    if idx == swe:
                        ts, te = self.token_location(subj_token_collect, subj_tokens)
                        tokens = subj_token_collect[:ts] + ['@'] + ['*'] + [subj_type] + ['*'] + subj_tokens + ['@'] + subj_token_collect[te+1:]
                        sents.extend(tokens)

                if ows <= idx and idx <= owe:
                    obj_token_collect.extend(tokens)
                    if idx == owe:
                        ts, te = self.token_location(obj_token_collect, obj_tokens)
                        tokens = obj_token_collect[:ts] + ["#"] + ['^'] + [obj_type] + ['^'] + obj_tokens + ["#"] + obj_token_collect[te+1:]
                        sents.extend(tokens)

    def read(self, file_in):
        features = []
        with open(file_in, "r") as fh:
            data = pd.read_csv(fh)

        for _, d in tqdm(data.iterrows()):
            ss, se = int(d['subject_start_idx']), int(d['subject_end_idx'])
            os, oe = int(d['object_start_idx']), int(d['object_end_idx'])
            input_ids, new_ss, new_os = self.tokenize(d['sentence'],d['subject_word'],d['object_word'], d['subject_type'], d['object_type'], ss, se, os, oe)
            rel = self.LABEL_TO_ID[d['label']]

            feature = {
                'input_ids': input_ids,
                'labels': rel,
                'ss': new_ss,
                'os': new_os,
            }
            features.append(feature)

        return features
    


In [148]:
sents

['〈',
 'So',
 '##me',
 '##th',
 '##ing',
 '〉',
 '는',
 '#',
 '^',
 '[PER]',
 '^',
 '조지',
 '해리',
 '##슨',
 '#',
 '##이',
 '쓰',
 '##고',
 '@',
 '*',
 '[ORG]',
 '*',
 '비틀즈',
 '@',
 '##가',
 '1969',
 '##년',
 '앨범',
 '《',
 'Ab',
 '##be',
 '##y',
 'Ro',
 '##ad',
 '》',
 '에',
 '담',
 '##은',
 '노래',
 '##다',
 '.']

In [141]:
words[2:]

['해리슨이', '쓰고', '비틀즈가', '1969년', '앨범', '《Abbey', 'Road》에', '담은', '노래다.']

In [74]:
cs=0
sents=[]
for word in sentence.split():
    tokens = tokenizer.tokenize(word)
    

    if cs <= ss and ss <= cs +len(word) -1:
        

    if cs == ss:
        new_ss = len(sents)
        sents.extend(['@'] + ['*'] + [subj_type] + ['*'])
        
        if cs + len(word) - 1 >= se:
            subtokens = tokenizer.tokenize(sub_entity)
            tokens_left =[t for t in tokens if t not in subtokens]
            sents.extend(subtokens + ['@'] + tokens_left) 
            
        else:
            sents.extend(subtokens)
    

        


    sents.extend(tokens)
    cs += len(word) + 1
    

SyntaxError: unexpected EOF while parsing (<ipython-input-74-b960057010d4>, line 1)

In [ ]:
    def tokenize(self, sentence, subj_type, obj_type, ss, se, os, oe):
        sents = []
        subj_type , obj_type = f"[{subj_type}]", f"[{obj_type}]"
        tokens = self.tokenizer.tokenize(sentence)
        for i_t, token in enumerate(tokens):
            if i_t == ss:
                new_ss = len(sents)
                sents.extend(['@'] + ['*'] + [subj_type] + ['*'])
            if i_t == se:
                sents.extend(['@'])
            if i_t == os:
                new_os = len(sents)
                sents.extend(["#"] + ['^'] + [obj_type] + ['^'])
            if i_t == oe:
                sents.extend(['#'])
            sents.append(token)
            print(sents)
        sents = sents[:self.args.model.max_seq_length - 2]
        input_ids = self.tokenizer.convert_tokens_to_ids(sents)
        input_ids = self.tokenizer.build_inputs_with_special_tokens(input_ids)
        
        return input_ids, new_ss + 1, new_os + 1


In [75]:
[1,2,3]

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [87]:
# 분할된 문장 리스트
words = ['〈Something〉는', '조지', '해리슨이', '쓰고', '비틀즈가', '1969년', '앨범', '《Abbey', 'Road》에', '담은', '노래다.']

# 원본 문장의 13번째 및 15번째 인덱스에 해당하는 문자를 찾아야 함
# 원본 문장은 모든 단어가 합쳐진 형태
original_sentence = " ".join(words)

# 13번째 및 15번째 인덱스의 문자를 찾음
thirteenth_char = original_sentence[13]
fifteenth_char = original_sentence[15]

# 이제 이 문자들을 포함하는 단어를 단어 리스트에서 찾아야 함
words_with_chars = [word for word in words if thirteenth_char in word or fifteenth_char in word]

thirteenth_char, fifteenth_char, words_with_chars



('조', ' ', ['조지'])

In [86]:
original_sentence

'〈Something〉는조지해리슨이쓰고비틀즈가1969년앨범《AbbeyRoad》에담은노래다.'